In [ ]:
!pip install PySimpleGUI
pip install googletrans==4.0.0-rc1
pip install pyperclip
pip install pillow
pip install gtts
pip install pyttsx3
pip install gTTS pygame
import PySimpleGUI as sg
from googletrans import Translator
from gtts import gTTS
import pygame
import tempfile
import threading

def create_layout():
    return [
        [sg.Text('Enter the text to translate:', font=('Helvetica', 12))],
        [sg.Multiline(key='text_input', size=(40, 5))],
        [sg.Text('Target Language:', font=('Helvetica', 12)), sg.Combo(list(language_mapping.keys()), key='target_language', default_value='')],
        [sg.Button('Translate', key='translate_button'), sg.Button('Clear', key='clear_button'), sg.Button('Copy to Clipboard', key='copy_button', disabled=True), sg.Button('Read Aloud', key='read_aloud_button', disabled=True), sg.Button('Stop Speaking', key='stop_speaking_button', disabled=True)],
        [sg.Text('Translated Text:', font=('Helvetica', 12))],
        [sg.Multiline(key='result_output', size=(40, 7), disabled=True)],
    ]

def translate_threaded():
    text_to_translate = values['text_input']
    target_language = language_mapping.get(values['target_language'], 'en')
    
    window['translate_button'].update(disabled=True)
    
    translation_thread = threading.Thread(target=perform_translation_with_retry, args=(text_to_translate, target_language))
    translation_thread.start()


def perform_translation_with_retry(text, target_language, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            source_language = 'fr' if target_language == 'en' else 'auto'
            translation = translator.translate(text, dest=target_language, src=source_language)
            translated_text = translation.text
            window['result_output'].update(value=translated_text)
            window['translate_button'].update(disabled=False)
            window['copy_button'].update(disabled=False)
            window['read_aloud_button'].update(disabled=False)
            window['stop_speaking_button'].update(disabled=True)
            return  # Successful translation, exit the loop
        except Exception as e:
            sg.popup_error(f"Translation error: {str(e)}")
            translated_text = "Translation error"
            retries += 1
            print(f"Retrying translation (Attempt {retries}/{max_retries})...")
    
    sg.popup_error("Failed to translate after multiple attempts. Check your internet connection.")
    window['result_output'].update(value=translated_text)
    window['translate_button'].update(disabled=False)
    window['copy_button'].update(disabled=False)
    window['read_aloud_button'].update(disabled=False)
    window['stop_speaking_button'].update(disabled=True)



def clear_text():
    window['text_input'].update(value='')
    window['result_output'].update(value='')
    window['copy_button'].update(disabled=True)
    window['read_aloud_button'].update(disabled=True)
    window['stop_speaking_button'].update(disabled=True)  # Disable 'Stop Speaking' button after clearing text

def copy_text():
    translated_text = window['result_output'].get()
    sg.clipboard_set(translated_text)
    sg.popup('Copied', 'The translated text has been copied to the clipboard.')

def read_aloud():
    translated_text = window['result_output'].get()
    speak_thread = threading.Thread(target=speak, args=(translated_text, language_mapping.get(values['target_language'], 'en')))
    speak_thread.start()

def stop_speaking():
    pygame.mixer.quit()
    window['stop_speaking_button'].update(disabled=True)  # Disable 'Stop Speaking' button after stopping speech

def speak(text, language_code):
    tts = gTTS(text, lang=language_code)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        temp_audio_path = temp_audio.name
        tts.save(temp_audio_path)
    
    pygame.mixer.init()
    pygame.mixer.music.load(temp_audio_path)
    pygame.mixer.music.play()
    
    window['read_aloud_button'].update(disabled=True)  # Disable 'Read Aloud' button while speaking
    window['stop_speaking_button'].update(disabled=False)  # Enable 'Stop Speaking' button while speaking
    
    while pygame.mixer.music.get_busy():
        if event == sg.WINDOW_CLOSED or values['stop_speaking_button']:
            pygame.mixer.music.stop()
            break
        pygame.time.Clock().tick(10)
    
    pygame.mixer.quit()
    
    temp_audio.close()
    window['read_aloud_button'].update(disabled=False)  # Enable 'Read Aloud' button after speaking

translator = Translator()

language_mapping = {
    "Auto Detect": "auto",
    "French": "fr",
    "English": "en",
    "Spanish": "es",
    "German": "de",
    "Arabic": "ar",
    "Chinese (Simplified)": "zh-cn",
    "Japanese": "ja",
    "Russian": "ru",
    "Korean": "ko",
    # Add more languages as needed
}

layout = create_layout()

sg.theme('LightGreen')

window = sg.Window('Translator', layout)

while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED:
        break
    elif event == 'translate_button':
        translate_threaded()
    elif event == 'clear_button':
        clear_text()
    elif event == 'copy_button':
        copy_text()
    elif event == 'read_aloud_button':
        read_aloud()
    elif event == 'stop_speaking_button':
        stop_speaking()

window.close()